In [2]:
import numpy as np
import pandas as pd
import nltk

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
df = pd.read_csv('all.csv')

In [4]:
df.columns

Index(['author', 'content', 'poem name', 'age', 'type'], dtype='object')

In [5]:
author = df['author']
content = df['content']

In [6]:
import re

def clean(string):
    for char in ['\r', '\n', '-', ',', '.', ';', '(', ')']:
        string = string.replace(char, ' ')
    return re.sub('\s+', ' ', string)

In [6]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()
content = content.apply(lambda x: ps.stem(clean(x)))

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(content)

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(author)

In [9]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=150)
X = svd.fit_transform(X)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rfc = RandomForestClassifier(n_estimators=150, criterion='entropy')
rfc.fit(X_train, y_train)
if X_test.shape[0]:
   print(accuracy_score(rfc.predict(X_test), y_test))   

0.5391304347826087


In [23]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=1000)
mlp.fit(X_train, y_train)
if X_test.shape[0]:
   print(accuracy_score(mlp.predict(X_test), y_test))

0.5652173913043478


In [24]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf', C=2, gamma=2)
svm.fit(X_train, y_train)
if X_test.shape[0]:
   print(accuracy_score(svm.predict(X_test), y_test))

0.591304347826087


In [49]:
from sklearn.ensemble import VotingClassifier

estimators = [
           ('svm', svm),
           ('mlp', mlp),
           ('rfc', rfc)
]

vc = VotingClassifier(estimators=estimators, voting='hard')
vc.fit(X_train, y_train)
if X_test.shape[0]:
   print(accuracy_score(vc.predict(X_test), y_test))

0.5826086956521739


In [52]:
t = """
Sir Charles into my chamber coming in,
When I was writing of my Fairy Queen;
I praysaid hewhen Queen Mab you do see
Present my service to her Majesty:
And tell her I have heard Fame's loud report
"""

le.inverse_transform(vc.predict(svd.transform(vectorizer.transform([ps.stem(clean(t))]))))

array(['DUCHESS OF NEWCASTLE MARGARET CAVENDISH'], dtype=object)

In [1]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
     ('vec', vectorizer),
     ('svd', svd),
     ('vc', vc)
])

NameError: name 'vectorizer' is not defined